In [1]:
# pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer

In [3]:
#Reading both CSVs and merging them
df_mar = pd.read_csv('mar22_unicorn.csv')
df_nov = pd.read_csv('nov22_unicorn.csv')
df = df_mar.merge(df_nov, left_on='Company', right_on='Company')
df.head()

,Company,Valuation,Date Joined_x,Industry_x,City_x,Country/Region,Continent,Year Founded,Funding,Select Investors_x,Unnamed: 0,Valuation ($B),Date Joined_y,Country,City_y,Industry_y,Select Investors_y
0,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",1,$127,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
1,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",2,$100,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China..."
2,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",3,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
3,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",98,$6.7,12/12/2011,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita..."
4,Canva,40,2018-01-08,Internet software & services,Surry Hills,Australia,Oceania,2012,$572M,"Sequoia Capital China, Blackbird Ventures, Mat...",4,$40,1/8/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat..."


In [4]:
#Combining duplicate data
df['Date Joined'] = np.where(df['Date Joined_x'] is None, df['Date Joined_y'], df['Date Joined_x'])
df['Industry'] = np.where(df['Industry_x'] is None, df['Industry_y'], df['Industry_x'])
df['City'] = np.where(df['City_x'] is None, df['City_y'], df['City_x'])
df['Country'] = np.where(df['Country'] is None, df['Country/Region'], df['Country'])

#Renaming valuation for both tables and converting numbers to int
df['Valuation ($B)'].replace('[$]', '', regex=True, inplace=True)
df['Valuation (November) ($B)'] = df['Valuation ($B)'].apply(float)
df['Valuation (March) ($B)'] = df['Valuation']

In [5]:
#Slicing and removing unused data
df = df[['Company','Date Joined',
       'Industry', 'Country', 'City', 'Continent', 'Year Founded', 'Funding',
       'Valuation (March) ($B)', 'Select Investors_x', 'Valuation (November) ($B)', 'Select Investors_y']]
df.rename(columns = {'Select Investors_x': 'Select Investors (March)', 'Select Investors_y': 'Select Investors (November)'}, inplace = True)
df.head()

,Company,Date Joined,Industry,Country,City,Continent,Year Founded,Funding,Valuation (March) ($B),Select Investors (March),Valuation (November) ($B),Select Investors (November)
0,SpaceX,2012-12-01,Other,United States,Hawthorne,North America,2002,$7B,100,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.0,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
1,SHEIN,2018-07-03,E-commerce & direct-to-consumer,China,Shenzhen,Asia,2008,$2B,100,"Tiger Global Management, Sequoia Capital China...",100.0,"Tiger Global Management, Sequoia Capital China..."
2,Stripe,2014-01-23,FinTech,United States,San Francisco,North America,2010,$2B,95,"Khosla Ventures, LowercaseCapital, capitalG",95.0,"Khosla Ventures, LowercaseCapital, capitalG"
3,Klarna,2011-12-12,Fintech,Sweden,Stockholm,Europe,2005,$4B,46,"Institutional Venture Partners, Sequoia Capita...",6.7,"Institutional Venture Partners, Sequoia Capita..."
4,Canva,2018-01-08,Internet software & services,Australia,Surry Hills,Oceania,2012,$572M,40,"Sequoia Capital China, Blackbird Ventures, Mat...",40.0,"Sequoia Capital China, Blackbird Ventures, Mat..."


In [6]:
df['Years to Unicorn'] = df['Date Joined'].apply(lambda x: int(x[:4])) - df['Year Founded']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Company                      1039 non-null   object 
 1   Date Joined                  1039 non-null   object 
 2   Industry                     1039 non-null   object 
 3   Country                      1039 non-null   object 
 4   City                         1022 non-null   object 
 5   Continent                    1039 non-null   object 
 6   Year Founded                 1039 non-null   int64  
 7   Funding                      1039 non-null   object 
 8   Valuation (March) ($B)       1039 non-null   int64  
 9   Select Investors (March)     1039 non-null   object 
 10  Valuation (November) ($B)    1039 non-null   float64
 11  Select Investors (November)  1038 non-null   object 
 12  Years to Unicorn             1039 non-null   int64  
dtypes: float64(1), int

In [7]:
df['Most Recent Valuation'] = np.where(df['Valuation (November) ($B)'] is None, df['Valuation (March) ($B)'], df['Valuation (November) ($B)'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Company                      1039 non-null   object 
 1   Date Joined                  1039 non-null   object 
 2   Industry                     1039 non-null   object 
 3   Country                      1039 non-null   object 
 4   City                         1022 non-null   object 
 5   Continent                    1039 non-null   object 
 6   Year Founded                 1039 non-null   int64  
 7   Funding                      1039 non-null   object 
 8   Valuation (March) ($B)       1039 non-null   int64  
 9   Select Investors (March)     1039 non-null   object 
 10  Valuation (November) ($B)    1039 non-null   float64
 11  Select Investors (November)  1038 non-null   object 
 12  Years to Unicorn             1039 non-null   int64  
 13  Most Recent Valuat

In [8]:
df['Industry'] = df['Industry'].apply(lambda x: ' '.join(word.capitalize() for word in x.split()))
df['Industry'] = df['Industry'].replace('&', 'And', regex=True)
df['Industry'].unique()

array(['Other', 'E-commerce And Direct-to-consumer', 'Fintech',
       'Internet Software And Services',
       'Supply Chain, Logistics, And Delivery', 'Consumer And Retail',
       'Data Management And Analytics', 'Edtech', 'Health',
       'Artificial Intelligence', 'Auto And Transportation', 'Hardware',
       'Travel', 'Cybersecurity', 'Mobile And Telecommunications'],
      dtype=object)

In [9]:
# df.to_csv('new_unicorn_data.csv')

In [10]:
# Define a function to extract numeric values
def extract_numeric_value(value):
    match = re.search(r'\$(\d+(\.\d+)?)([BM]?)', str(value))
    if match:
        number = float(match.group(1))
        suffix = match.group(3)
        if suffix == 'B':
            number = int(number) if number.is_integer() else round(number, 3)
        elif suffix == 'M':
            number *= 0.001 
        return number
    else:
        return None

# Applying the function to convert the values in Funding
df['Funding_Numeric ($B)'] = df['Funding'].apply(extract_numeric_value)

# Calculate the mean of the numeric values
mean_value = df['Funding_Numeric ($B)'].mean()

# Use SimpleImputer to replace NaN values with the mean
imputer = SimpleImputer(strategy='constant', fill_value=mean_value)
df['Funding_Numeric ($B)'] = imputer.fit_transform(df[['Funding_Numeric ($B)']])

df["Funding_Numeric ($B)"].head()


0    7.000
1    2.000
2    2.000
3    4.000
4    0.572
Name: Funding_Numeric ($B), dtype: float64

In [11]:
df.head()

,Company,Date Joined,Industry,Country,City,Continent,Year Founded,Funding,Valuation (March) ($B),Select Investors (March),Valuation (November) ($B),Select Investors (November),Years to Unicorn,Most Recent Valuation,Funding_Numeric ($B)
0,SpaceX,2012-12-01,Other,United States,Hawthorne,North America,2002,$7B,100,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.0,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,127.0,7.000
1,SHEIN,2018-07-03,E-commerce And Direct-to-consumer,China,Shenzhen,Asia,2008,$2B,100,"Tiger Global Management, Sequoia Capital China...",100.0,"Tiger Global Management, Sequoia Capital China...",10,100.0,2.000
2,Stripe,2014-01-23,Fintech,United States,San Francisco,North America,2010,$2B,95,"Khosla Ventures, LowercaseCapital, capitalG",95.0,"Khosla Ventures, LowercaseCapital, capitalG",4,95.0,2.000
3,Klarna,2011-12-12,Fintech,Sweden,Stockholm,Europe,2005,$4B,46,"Institutional Venture Partners, Sequoia Capita...",6.7,"Institutional Venture Partners, Sequoia Capita...",6,6.7,4.000
4,Canva,2018-01-08,Internet Software And Services,Australia,Surry Hills,Oceania,2012,$572M,40,"Sequoia Capital China, Blackbird Ventures, Mat...",40.0,"Sequoia Capital China, Blackbird Ventures, Mat...",6,40.0,0.572


In [12]:
# Move 'Funding_Numeric' next to 'Funding'
columns = list(df.columns)

columns.insert(columns.index('Funding') + 1, columns.pop(columns.index('Funding_Numeric ($B)')))

df = df[columns]

# Save the DataFrame back to a CSV file
df.to_csv('new_unicorn_data_updated.csv', index=False)

In [13]:
df.head()

,Company,Date Joined,Industry,Country,City,Continent,Year Founded,Funding,Funding_Numeric ($B),Valuation (March) ($B),Select Investors (March),Valuation (November) ($B),Select Investors (November),Years to Unicorn,Most Recent Valuation
0,SpaceX,2012-12-01,Other,United States,Hawthorne,North America,2002,$7B,7.000,100,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.0,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,127.0
1,SHEIN,2018-07-03,E-commerce And Direct-to-consumer,China,Shenzhen,Asia,2008,$2B,2.000,100,"Tiger Global Management, Sequoia Capital China...",100.0,"Tiger Global Management, Sequoia Capital China...",10,100.0
2,Stripe,2014-01-23,Fintech,United States,San Francisco,North America,2010,$2B,2.000,95,"Khosla Ventures, LowercaseCapital, capitalG",95.0,"Khosla Ventures, LowercaseCapital, capitalG",4,95.0
3,Klarna,2011-12-12,Fintech,Sweden,Stockholm,Europe,2005,$4B,4.000,46,"Institutional Venture Partners, Sequoia Capita...",6.7,"Institutional Venture Partners, Sequoia Capita...",6,6.7
4,Canva,2018-01-08,Internet Software And Services,Australia,Surry Hills,Oceania,2012,$572M,0.572,40,"Sequoia Capital China, Blackbird Ventures, Mat...",40.0,"Sequoia Capital China, Blackbird Ventures, Mat...",6,40.0


In [16]:
# Test: Filter rows where Funding_Numeric is less than 1
filtered_df = df[df['Funding_Numeric ($B)'] < 1]
print(filtered_df)

          Company Date Joined                           Industry  \
4           Canva  2018-01-08     Internet Software And Services   
16    Xiaohongshu  2016-03-31  E-commerce And Direct-to-consumer   
17           Miro  2022-01-05     Internet Software And Services   
19          Rapyd  2019-12-03                            Fintech   
20        Discord  2018-04-20     Internet Software And Services   
...           ...         ...                                ...   
1034     Zhaogang  2017-06-29  E-commerce And Direct-to-consumer   
1035  Zhuan Zhuan  2017-04-18  E-commerce And Direct-to-consumer   
1036     Zihaiguo  2021-05-06                Consumer And Retail   
1037         Zopa  2021-10-19                            Fintech   
1038        Zwift  2020-09-16  E-commerce And Direct-to-consumer   

             Country           City      Continent  Year Founded Funding  \
4          Australia    Surry Hills        Oceania          2012   $572M   
16             China       Shan